In [3]:
import pandas as pd
# import operators as op
import numpy as np
import random
import gc
import tensorflow as tf
# import lightgbm as lgbm
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.models import model_from_json
from sklearn import linear_model
# For neural network
from tensorflow.keras.models import Sequential, Model, model_from_json
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Dense, Activation, LeakyReLU, BatchNormalization, PReLU, Dropout
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, CSVLogger, EarlyStopping
from keras.constraints import maxnorm
from keras import regularizers, optimizers, losses
##### to limit number of cores used by TensorFlow ####
from keras import backend as K
import tensorflow 
import keras
import matplotlib.pyplot as plt


In [77]:
# from IPython.display import display, clear_output


In [56]:
def getNNModel3(input_dim=9):
    
    model = Sequential()
    model.add(Dense(16, input_dim=input_dim, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(8, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(26, activation='linear'))
    return model

In [57]:
def compile1(model):
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mse'])

In [4]:
model = getNNModel3(26+4)
compile1(model)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 16)                496       
_________________________________________________________________
dense_1 (Dense)              (None, 32)                544       
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 264       
_________________________________________________________________
dense_3 (Dense)              (None, 26)                234       
Total params: 1,538
Trainable params: 1,538
Non-trainable params: 0
_________________________________________________________________


In [5]:
def encodeWord(word, guesses):
    encoding = []
    for c in word:
        ci = ord(c)-ord('a')
        if(guesses[ci]==0):
            encoding += [27]
        else:
            encoding += [ord(c)-ord('a')+1]
    return np.concatenate([np.array(encoding), guesses])

In [83]:
encodeWord("asdas",np.zeros(26))

array([27., 27., 27., 27., 27.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.])

In [7]:
words = []
with open('words_250000_train.txt','r') as f:
    for l in f.readlines():
        words += [l[:-1]]

In [8]:
words_by_length = {}
for w in words:
    if(len(w) in words_by_length.keys()):
        words_by_length[len(w)] += [w]
    else:
        words_by_length[len(w)] = [w]

In [9]:
words_by_length.keys()

dict_keys([3, 6, 4, 5, 8, 7, 10, 9, 11, 12, 13, 15, 14, 20, 17, 16, 2, 21, 18, 19, 25, 22, 1, 23, 29, 24, 28, 27])

In [49]:
def getValidGuesses(word, guesses):
    valids = np.zeros(26)
    for c in word:
        ci = ord(c) - ord('a')
        if(guesses[ci]==0):
            valids[ci]+=1
    return valids/(sum(valids)+1)
            

In [84]:
getValidGuesses("aabcdef",np.zeros(26))

array([0.25 , 0.125, 0.125, 0.125, 0.125, 0.125, 0.   , 0.   , 0.   ,
       0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ])

In [12]:
previous_guesses = np.zeros(26)


In [39]:
models_all = {}
for k in words_by_length.keys():
    print(k)
    models_all[k] = getNNModel3(k+26)
    compile1(models_all[k])
    

3
6
4
5
8
7
10
9
11
12
13
15
14
20
17
16
2
21
18
19
25
22
1
23
29
24
28
27


In [40]:
models_all[5].summary()

Model: "sequential_60"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_240 (Dense)            (None, 16)                512       
_________________________________________________________________
dense_241 (Dense)            (None, 32)                544       
_________________________________________________________________
dense_242 (Dense)            (None, 8)                 264       
_________________________________________________________________
dense_243 (Dense)            (None, 26)                234       
Total params: 1,554
Trainable params: 1,554
Non-trainable params: 0
_________________________________________________________________


In [79]:
gc.collect()

15060

In [44]:
valid_guesses

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.33333333,
       0.        , 0.        , 0.33333333, 0.        , 0.33333333,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        ])

In [81]:
np.zeros(2,3)

TypeError: Cannot interpret '3' as a data type

In [78]:
models_all[5] = getNNModel3(31)
compile1(models_all[5])

In [ ]:
#######single word at a time, single fit at a time#####
# word_length = 5
# epochs = 5
# for e in range(epochs):
#         print('epoch: ',e)
#         wins = 0
#         losses = 0
#         prev_start_index = 0
#         block_size = 10
        
#         for wi,word in enumerate(words_by_length[word_length][prev_start_index:prev_start_index+block_size]):
# #             if(wi%10000==0):
# # #                 clear_output(,wait=True)
# # #                 display('Iteration '+str(i)+' Score: '+str(uniform(0, 1)))
# #                 print(wi)
# #             print(word)
#             tries_left = np.zeros(block_size)+6
#             guesses = np.zeros((block_size,26))
#             valid_guesses = np.array([getValidGuesses(word,guesses)])
#             while(tries_left!=0 and sum(valid_guesses)!=0):
#                 encoded = encodeWord(word, guesses).reshape(1,-1)
#                 predicted = models_all[word_length].predict([encoded])

#                 models_all[word_length].fit(encoded,valid_guesses.reshape(1,-1), verbose=False)

#     #             print(guesses)

#                 predicted[0][(guesses!=0)] = 0
#                 new_guess = np.argmax(predicted)

#                 guesses[new_guess] = 1

#                 if(valid_guesses[new_guess]>0):
#                     valid_guesses = getValidGuesses(word, guesses)
#                 else:
#                     tries_left-=1

#             if(tries_left==0):
#                 losses += 1
# #                 print('valid guesses left:',np.where(valid_guesses!=0),[chr(ord('a')+i) for i in np.where(valid_guesses!=0)[0]])
# #                 print('tries_left: ',tries_left)
#             else:
#                 wins += 1
# #                 print('tries_left: ',tries_left)
#         prev_start_index += block_size
#         print('wins: ',wins,"and losses:",losses)
# #                 print('guessed:',chr(ord('a')+new_guess)+" and tried_left:",tries_left)
# #                 print('valid guesses left:',[chr(ord('a')+i) for (i,x) in enumerate(valid_guesses)  if x>0])



In [93]:
np.random.choice(words_by_length[word_length][:100000],1000)

array(['nicol', 'fulup', 'donia', 'kaine', 'genys', 'arcos', 'gapes',
       'goudy', 'nobby', 'gough', 'creon', 'givin', 'deter', 'claye',
       'boosy', 'decem', 'bakra', 'azide', 'pamir', 'toled', 'tulip',
       'troot', 'segni', 'mosts', 'astto', 'neeps', 'sluit', 'mazda',
       'yanky', 'ranid', 'hexad', 'iloko', 'soncy', 'unman', 'kelcy',
       'oshee', 'paint', 'eaten', 'idmon', 'plott', 'schug', 'ociaa',
       'grise', 'estop', 'oakie', 'bombo', 'boogy', 'redip', 'adron',
       'ovapa', 'masai', 'venez', 'polio', 'tepal', 'orvas', 'acast',
       'sugat', 'gwari', 'wodgy', 'issue', 'abram', 'tofus', 'feune',
       'ossie', 'email', 'algin', 'abohm', 'winny', 'charm', 'muons',
       'brays', 'balpa', 'shojo', 'huaco', 'lotis', 'pfalz', 'sobor',
       'grike', 'galli', 'rebag', 'cappy', 'roana', 'ruffs', 'ejido',
       'taces', 'ligon', 'jumpy', 'musil', 'chirm', 'biron', 'giing',
       'troff', 'orosi', 'needy', 'ohare', 'kaphs', 'kalle', 'oshee',
       'skint', 'zaw

In [97]:
traning_set5 = 
test_set5 = [x for x in words_by_length[5] if x not in traning_set5]

In [107]:
#######single word at a time, multiple fit at a time#####
epochs = 5
for e in range(epochs):
    for word_length in list(sorted(words_by_length.keys()))[:4]:

            training_set = np.random.choice(words_by_length[word_length],int(0.8*len(words_by_length[word_length])))
            test_set = [x for x in words_by_length[word_length] if x not in training_set]
    
            print('word_lengthe: ',word_length)
            wins = 0
            losses = 0
            gc.collect()
            for wi,word in enumerate(np.random.choice(traning_set5,10)):
#                 if(wi%100==0):
#                     print(wi)
                tries_left = 6
                guesses = np.zeros(26)
                valid_guesses = getValidGuesses(word,guesses)

                encoded_all = []
                valid_guesses_all = []

                while(tries_left!=0 and sum(valid_guesses)!=0):
                    encoded = encodeWord(word, guesses)

                    encoded_all += [encoded]
                    valid_guesses_all += [valid_guesses]

                    predicted = models_all[word_length].predict([encoded.reshape(1,-1)])
                    predicted[0][(guesses!=0)] = 0
                    new_guess = np.argmax(predicted)

                    guesses[new_guess] = 1

                    if(valid_guesses[new_guess]>0):
                        valid_guesses = getValidGuesses(word, guesses)
                    else:
                        tries_left-=1

                encoded_all += [encodeWord(word, guesses)]
                valid_guesses_all += [valid_guesses]



                encoded_all = np.array(encoded_all)
                valid_guesses_all = np.array(valid_guesses_all)


    #             print(encoded_all.shape, valid_guesses_all.shape)

                models_all[word_length].fit(encoded_all,valid_guesses_all, verbose=False)        

                if(tries_left==0):
                    losses += 1
    #                 print('valid guesses left:',np.where(valid_guesses!=0),[chr(ord('a')+i) for i in np.where(valid_guesses!=0)[0]])
    #                 print('tries_left: ',tries_left)
                else:
                    wins += 1
    #                 print('tries_left: ',tries_left)
            prev_start_index += block_size
            print('wins: ',wins,"and losses:",losses)

        
#                 print('guessed:',chr(ord('a')+new_guess)+" and tried_left:",tries_left)
#                 print('valid guesses left:',[chr(ord('a')+i) for (i,x) in enumerate(valid_guesses)  if x>0])



word_lengthe:  1


ValueError: Error when checking input: expected dense_316_input to have shape (27,) but got array with shape (31,)

In [80]:
#######multiple word at a time#####
# word_length = 5
# epochs = 5
# for e in range(epochs):
#         print('epoch: ',e)
#         wins = 0
#         losses = 0
#         prev_start_index = 0
#         block_size = 10
        
#         for wi in range(10):
# #             if(wi%10000==0):
# # #                 clear_output(,wait=True)
# # #                 display('Iteration '+str(i)+' Score: '+str(uniform(0, 1)))
# #                 print(wi)
# #             print(word)
#             tries_left = np.zeros(block_size)+6
#             guesses = np.zeros((block_size,26))
#             valid_guesses = np.array([getValidGuesses(word,guesses) for word in words_by_length[word_length][prev_start_index+block_size*wi: prev_start_index+block_size*(wi+1)]])
#             while(tries_left!=0 and sum(valid_guesses)!=0):
#                 encoded = encodeWord(word, guesses).reshape(1,-1)
#                 predicted = models_all[word_length].predict([encoded])

#                 models_all[word_length].fit(encoded,valid_guesses.reshape(1,-1), verbose=False)

#     #             print(guesses)

#                 predicted[0][(guesses!=0)] = 0
#                 new_guess = np.argmax(predicted)

#                 guesses[new_guess] = 1

#                 if(valid_guesses[new_guess]>0):
#                     valid_guesses = getValidGuesses(word, guesses)
#                 else:
#                     tries_left-=1

#             if(tries_left==0):
#                 losses += 1
# #                 print('valid guesses left:',np.where(valid_guesses!=0),[chr(ord('a')+i) for i in np.where(valid_guesses!=0)[0]])
# #                 print('tries_left: ',tries_left)
#             else:
#                 wins += 1
# #                 print('tries_left: ',tries_left)
#         prev_start_index += block_size
#         print('wins: ',wins,"and losses:",losses)
# #                 print('guessed:',chr(ord('a')+new_guess)+" and tried_left:",tries_left)
# #                 print('valid guesses left:',[chr(ord('a')+i) for (i,x) in enumerate(valid_guesses)  if x>0])



epoch:  0
0


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


KeyboardInterrupt: 

In [96]:
for i in sorted(words_by_length.keys()):
    print(i,len(words_by_length[i]))

1 17
2 264
3 2201
4 5287
5 11274
6 19541
7 25948
8 30452
9 30906
10 26953
11 22786
12 18178
13 12956
14 8710
15 5211
16 3143
17 1775
18 859
19 441
20 225
21 98
22 44
23 14
24 9
25 3
27 2
28 1
29 2


In [104]:
########testing###########
def getAccuracy(word_length):
    wins = 0
    losses = 0
    for word in test_set5[:1000]:
        tries_left = 6
        guesses = np.zeros(26)
        valid_guesses = getValidGuesses(word,guesses)
        while(tries_left!=0 and sum(valid_guesses)!=0):
            encoded = encodeWord(word, guesses).reshape(1,-1)
            predicted = models_all[word_length].predict([encoded])

    #         models_all[word_length].fit(encoded,valid_guesses.reshape(1,-1), verbose=False)
    # 
    #             print(guesses)

            predicted[0][(guesses!=0)] = 0
            new_guess = np.argmax(predicted)

            guesses[new_guess] = 1

            if(valid_guesses[new_guess]>0):
                valid_guesses = getValidGuesses(word, guesses)
            else:
                tries_left-=1

        if(tries_left==0):
            losses += 1
    #                 print('valid guesses left:',np.where(valid_guesses!=0),[chr(ord('a')+i) for i in np.where(valid_guesses!=0)[0]])
    #                 print('tries_left: ',tries_left)
        else:
            wins += 1

    print('wins: ',wins,"and losses:",losses)
    return wins/(wins+losses)

In [105]:
getAccuracy(5)

wins:  117 and losses: 883


0.117

In [16]:

for i in range(10):
    model.fit(np.random.rand(10,30), np.random.rand(10,26))

Train on 10 samples
10/10 [==============================] - 0s 8ms/sample - loss: 0.3638 - mean_squared_error: 0.3638
Train on 10 samples
10/10 [==============================] - 0s 669us/sample - loss: 0.3547 - mean_squared_error: 0.3547
Train on 10 samples
10/10 [==============================] - 0s 1ms/sample - loss: 0.3309 - mean_squared_error: 0.3309
Train on 10 samples
10/10 [==============================] - 0s 1ms/sample - loss: 0.3511 - mean_squared_error: 0.3511
Train on 10 samples
10/10 [==============================] - 0s 952us/sample - loss: 0.3035 - mean_squared_error: 0.3035
Train on 10 samples
10/10 [==============================] - 0s 547us/sample - loss: 0.3353 - mean_squared_error: 0.3353
Train on 10 samples
10/10 [==============================] - 0s 504us/sample - loss: 0.3059 - mean_squared_error: 0.3059
Train on 10 samples
10/10 [==============================] - 0s 457us/sample - loss: 0.3078 - mean_squared_error: 0.3078
Train on 10 samples
10/10 [===========